<div style="font-variant: small-caps; 
      font-weight: normal; 
      font-size: 40px; 
      text-align: center; 
      padding: 15px; 
      margin: 10px;">
  Chatbot - Génératif - Agnostique
  </div> 
  
  
<div style="font-variant: small-caps; 
      font-weight: normal; 
      font-size: 30px; 
      text-align: center; 
      padding: 15px; 
      margin: 10px;">
  M2DS Octobre 2019
  </div> 
  

  <div style="font-variant: small-caps; 
      font-weight: normal; 
      font-size: 20px; 
      text-align: center; 
      padding: 15px;">
  Jean-baptiste Aujogue
  </div>

In [1]:
from __future__ import unicode_literals, print_function, division
import os
from io import open
import unicodedata
import string
import re
import random
import pickle
import copy
# package a installer d'abord avec anaconda
#import spacy

import torch
import torch.nn as nn
from torch.autograd import Variable
from torch import optim
import torch.nn.functional as F
from torch.utils.data import DataLoader

# package a installer d'abord avec anaconda
import gensim
from gensim.models import KeyedVectors

#import nltk
#nltk.download()
#from nltk.tokenize import sent_tokenize, word_tokenize

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import numpy as np

import unidecode
from unidecode import unidecode

C:\ProgramData\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
import time
import math
import re
import json

In [3]:
%matplotlib inline

import warnings
warnings.filterwarnings("ignore")

USE_CUDA = torch.cuda.is_available()
print(USE_CUDA)
device = torch.device("cuda" if USE_CUDA else "cpu")

True


In [4]:
import sys
sys.path.append('C:\\Users\\Jb\\Desktop\\NLP\\chatNLP')

# 1 Training data preparation

### 1.1 Simple formatting of training dialogues

In [5]:
# -------------------- enlèvement des stopwords ---------------------------------------
def TrimWordsSentence(sentence, stopwords):
    '''Remove stopwords from a sentence'''
    resultwords = [word for word in sentence if word.lower() not in stopwords]
    return resultwords

def TrimWordsDialogue(dialogue, stopwords):
    '''Remove stopwords from user utterances in a dialogue'''
    for pair in dialogue: 
        pair[0] = TrimWordsSentence(pair[0], stopwords)
        #pair[1] = pair[1].strip()
    return dialogue

def TrimWords(dialogues, stopwords):
    '''Remove stopwords from user utterances in a list of dialogues'''
    return [TrimWordsDialogue(dialogue, stopwords) for dialogue in dialogues ]

    


# --------------------------- Normalisation -------------------------------
def normalizeString(s):
    '''Remove rare symbols from a string'''
    def unicodeToAscii(s):
        """Turn a Unicode string to plain ASCII, thanks to http://stackoverflow.com/a/518232/2809427"""
        return ''.join(c for c in unicodedata.normalize('NFD', s) if unicodedata.category(c) != 'Mn')
 
    s = unicodeToAscii(s.lower().strip())
    #s = re.sub(r"[^a-zA-Z0-9?&\%\-\_]+", r" ", s) 
    s = re.sub("\(", r" ( ", s)
    s = re.sub("\)", r" ) ", s)
    s = re.sub(r"\.", r" . ", s)
    s = re.sub(r",", r" , ", s)
    s = re.sub(r"!", r" ! ", s)
    s = re.sub(r":", r" : ", s)
    s = re.sub(r"-", r" - ", s)
    s = re.sub(r"'", r" ' ", s)
    s = re.sub(r";", r" ; ", s)
    s = re.sub(r' +', r' ', s).strip()
    return s 



#--------------------- import des dialogues --------------------
def importDialogues(fichier, limite = None):
    '''Import a textfile containing dialogues and returns a list, each element 
       corresponding to a dialogue and also being under the form of a list, with 
       each element being a list of two elements : an element giving a user 
       utterance and another element giving the bot response. Both elements are 
       normalized strings.
       Ex. The dialogue :
       
               hi    hello what can i help you with today
               can you book a table    i m on it
               
       now becomes :
       
              [['hi', 'hello what can i help you with today'], 
               ['can you book a table', 'i m on it']]
               
       Lines corresponding to user utterance with no bot response are discarted.
    '''
    def cleanS(s):
        cleans = normalizeString(s).split(' ')
        cleans = [word for word in cleans if word.strip() != '']
        return cleans
    dialogues_import = open(fichier, encoding='utf-8').read().strip().split('\n\n')
    dialogues = []
    for i, d in enumerate(dialogues_import):
        dialogue = []
        lines = d.split('\n')
        for l in lines:
            if len(l.split('\t')) == 2 :
                pair = [cleanS(s) for s in l.split('\t')]
                dialogue.append(pair)
            elif len(l.split('\t')) == 3 :
                pair = [cleanS(s) for s in l.split('\t')[:2]]
                dialogue.append(pair)
        dialogues.append(dialogue)
        if limite is not None and i == limite -1 :
            break

    return dialogues






#------------------ Dictionnaire des mots variables -----------------------------
def motVar(file):
    '''Applies to the Master's program dataset.
       Import the collection of pairs token-content for a set of variable words.
    '''
    lines = open(file, encoding='utf-8').read().strip().split('\n')
    motsVar = {}
    for l in lines :
        cle, valeur = l.split('\t')
        motsVar[cle.lower()] = valeur
    return motsVar




# ----------------------- Création de la liste des dialogues--------------------
def prepareData(opt):
    '''Import dialogue from text file and apply some formatting operations,
       as described in the functions 
               - importDialogues
               - modify
               - TrimWords
               - filterDialogues
    '''
    dialogues = importDialogues(fichier = opt['fichier'], 
                                limite = opt['limite'])
    dialogues = modify(dialogues) if opt['modify'] else dialogues
    dialogues = TrimWords(dialogues, opt['stopwords']) # on enlève les stopwords de chaque question
    print(" %s dialogues ..." % len(dialogues))
    print(dialogues[0])
    if opt['filtre'] :
        #for pair in [pair for pair in pairs if not filterPair(pair)]:
        #    print('%s (%d) -> %s (%d)' % (pair[0],len(pair[0].split()),pair[1],len(pair[1].split())))  
        dialogues = filterDialogues(dialogues, opt['max_length'])
        print('')
        print("... reduced to %s dialogues" % len(dialogues))

    return dialogues

In [6]:
motsVar = motVar('C:\\Users\\Jb\\Desktop\\NLP\\data\\M2DS\\chatbot-M2-DS-Variables.txt')

Turn each dialogue of the corpus into torch variables

In [7]:
def variableFromSentence(lang, sentence, rand = 0): # sentence = [str]
    indexes=[]
    unknowns = 0
    for word in sentence:
        p = random.random()
        if word not in lang.word2index.keys() and 'UNK' in lang.word2index.keys() :
            #pass
            indexes.append(lang.word2index['UNK'])
        elif p >= rand :
            indexes.append(lang.word2index[word])
        elif p < rand :
            e = random.choice([1, 2])
            if e == 1 :  # doesn't put any word
                pass
            elif e == 2 and 'UNK' in lang.word2index.keys() :# hide word with UNK_Token
                indexes.append(lang.word2index['UNK'])
    indexes.append(lang.word2index['EOS'])                                
    result = Variable(torch.LongTensor([[i] for i in indexes]))
    return result

def variableFromDialogue(lang, dialogue, rand = 0): # sentence = [str]
    result = []
    for paire in dialogue :
        el1 = variableFromSentence(lang, paire[0], rand = rand)
        el2 = variableFromSentence(lang, paire[1], rand = 0)
        result.append([el1, el2])
    return result

def variableFromAllDialogues(lang, dialogues, rand = 0): # sentence = [str]
    result = []
    for dialogue in dialogues :
        result.append(variableFromDialogue(lang, dialogue, rand))
    return result

### 1.2 Language class

In [8]:
class Lang:
    def __init__(self, name, init = 3):
        self.name = name
        if init == 0 :
            self.word2index = {}
            self.word2count = {}
            self.index2word = {}
            self.n_words = 0
        elif init == 3 :
            self.word2index = {"SOS": 0, "EOS": 1, "UNK": 2}
            self.word2count = {"SOS": 0, "EOS": 0, "UNK": 0}
            self.index2word = {0: "SOS", 1: "EOS", 2: "UNK"}
            self.n_words = 3

        
    def addWord(self, word):
        '''Add a word to the language'''
        if word not in self.word2index:
            if word.strip() != '' :
                self.word2index[word] = self.n_words
                self.word2count[word] = 1
                self.index2word[self.n_words] = word
                self.n_words += 1
        else:
            self.word2count[word] += 1
            
            
    def addSentence(self, sentence):
        '''Add to the language all words of a sentence'''
        if type(sentence) == str :
            for word in nltk.word_tokenize(sentence): # sentence.split()
                self.addWord(word)
        elif type(sentence) == list :
            for word in sentence:
                self.addWord(word)            
            
            
    def addDescriptions(self, descriptions, lvl = 1):
        '''Add to the language all words contained into : either all user utterances 
          (if i = 0) or all bot utterances (if i = 1), of a list of dialogues'''
        for description in descriptions :
            
            # si la description est une seule ligne
            if type(description) == str :
                    try :
                        if lvl == 2 :
                            for sentence in nltk.sent_tokenize(description) :
                                self.addSentence(sentence)
                        else :
                            self.addSentence(description)
                    except IndexError:
                        print("Problem with {}".format(description))
                        
            # si la description est une liste de lignes
            elif type(description) == list :
                for line in description:
                    try :
                        self.addSentence(line)
                    except IndexError:
                        print("Problem with {}".format(line))
                              
            # sinon
            else :
                print("Problem with {}".format(description))
            
            
    def addDialogues(self, dialogues, i):
        '''Add to the language all words contained into : either all user utterances 
          (if i = 0) or all bot utterances (if i = 1), of a list of dialogues'''
        for dialogue in dialogues :
            for pair in dialogue:
                try :
                    self.addSentence(pair[i])
                except IndexError:
                    print("Problem with {}".format(pair))

In [9]:
def generateLanguages(dialogues):
    '''Generate three languages classes out of a list of dialogues :
            - input_lang containing the user's vocabulary
            - output lang containing the bot vocabulary
            - output_sentence_lang containing the bot answers as words of a vocabulary
    '''
    lang = Lang('M2DS')
    
    lang.addDialogues(dialogues, 0)
    lang.addDialogues(dialogues, 1)
    print("Mots comptés :")
    print(lang.name, lang.n_words)
    
    return lang


def ajout(dialogues, lang, i= 1):
    '''addDialogues method of the Lang class with prints.'''
    lang.addDialogues(dialogues, i)
    print(lang.name, lang.n_words)
    return lang 


def ajoutSentences(dialogues, sentences_lang, i = 1) :
    '''Add sentences as words to a given language'''
    for dialogue in dialogues :
        for pair in dialogue :
            try :
                sentences_lang.addWord(pair[i])
            except IndexError:
                print("Problem with {}".format(pair))
                
    return sentences_lang

## 1.3 Master's program dataset

In [10]:
MAX_LENGTH = 100
max_length = MAX_LENGTH
stopwords = []

Master_train =   {'fichier': 'C:\\Users\\Jb\\Desktop\\NLP\\data\\M2DS\\M2DS_2019_10\\ChatbotDS_P_Train.tsv', #dialog-babi-task1-API-calls-trn.txt
                   'modify' : False,
                   'stopwords' : stopwords,
                   'max_length' : MAX_LENGTH ,
                   'limite' : None,
                   'filtre' : False}

dialogues_Master = prepareData(Master_train)

Master_test  =   {'lang1': 'lang_client',
                   'lang2' : 'lang_agent',
                   'fichier': 'C:\\Users\\Jb\\Desktop\\NLP\\data\\M2DS\\M2DS_2019_10\\ChatbotDS_P_Test.tsv', #dialog-babi-task1-API-calls-trn.txt
                   'modify' : False,
                   'stopwords' : stopwords,
                   'max_length' : MAX_LENGTH ,
                   'limite' : None,
                   'reverse' : False,
                   'filtre' : False}

dialogues_Master_test = prepareData(Master_test)

 10000 dialogues ...
[[['il', 'vaut', 'mieux', 'suivre', 'quel', 'parcours', '?'], ['les', 'deux', 'parcours', 'sont', 'equivalents', ',', 'c', "'", 'est', 'a', 'vous', 'de', 'faire', 'un', 'choix', '.']], [['quand', 'est', 'ce', 'que', 'commence', 'les', 'cours', '?'], ['la', 'rentree', 'est', 'le', 'daterentree']], [['comment', 'je', 'prend', 'contact', 'avec', 'la', 'scolarite', '?'], ['les', 'contacts', 'de', 'la', 'scolarite', 'sont', 'accessibles', 'sur', ':', 'sitescolarite']], [['est', 'ce', 'que', 'il', 'y', 'a', 'des', 'cours', 'en', 'anglais', '?'], ['plusieurs', 'supports', 'de', 'cours', 'sont', 'en', 'anglais']], [['tous', 'les', 'cours', 'sont', 'obligatoires', '?'], ['tous', 'les', 'cours', 'sont', 'obligatoires', ',', 'il', 'n', "'", 'existe', 'pas', 'd', "'", 'options', '.']], [['merci', 'pour', 'les', 'renseignements'], ['je', 'vous', 'en', 'prie']], [['peux', 'tu', 'm', "'", 'aider', '?'], ['je', 'suis', 'la', 'pour', 'vous', 'renseigner', 'sur', 'le', 'master', '2'

In [11]:
clean_sentences = [sentence for sentence in list(set([' '.join(pair[0]) for diag in dialogues_Master for pair in diag]))]
print(len(clean_sentences))

1362


In [12]:
lang_M2DS = generateLanguages(dialogues_Master)

Mots comptés :
M2DS 1298


In [13]:
# compute ONCE
#with open(r'C:\\Users\\Jb\\Desktop\\NLP\\saves\\lang\\lang_M2DS_2019_10.file', 'wb') as fileObject :
#    pickle.dump(lang_M2DS, fileObject)

# 1062 / 1272 / 1291 / 1298 mots comptés

In [14]:
def importLang(name):
    with open('C:\\Users\\Jb\\Desktop\\NLP\\saves\\lang\\'+name+'.file', 'rb') as fil :
        lang = pickle.load(fil)
    return lang

In [15]:
lang_M2DS = importLang('lang_M2DS_2019_10') # 1062 / 1272 / 1291 / 1298

# 2 Word Embedding Pretraining

## 2.1 Embedding Models

In [16]:
from chatNLP.models.Word_Embedding import CBOW, SkipGram, WordEmbeddingTrainer

### 2.1.1 Cbow

#### Data

In [17]:
def flatten(description) :
    '''Baisse le nombre de niveaux de 1 dans la description'''
    flatten = []
    for line in description :
        flatten += line
    return flatten


def generateCBOWdata(descriptions, context_size = 5) :
    '''descriptions = [[str]]'''
    data = []
    for description in descriptions :
        lines = flatten(description)
        line =  flatten(lines)
        line = ['SOS' for i in range(context_size)] + line + ['EOS' for i in range(context_size)] 
        for i in range(context_size, len(line) - context_size):
            context = line[i-context_size : i] + line[i+1 : i+context_size+1]
            target = [line[i]]
            data.append((context, target))
    return data

#### Model

In [16]:
cbow = CBOW(device,
            lang = lang_M2DS,
            context_size = 5,
            embedding_dim = 100,
            hidden_dim = 100)
cbow = cbow.to(device)

In [ ]:
cbow.load_state_dict(torch.load('C:\\Users\\Jb\\Desktop\\NLP\\saves\\models\\M2DS_cbow.pth'))

### 2.1.2 SkipGram

**Data**

In [18]:
def flatten(description) :
    '''Baisse le nombre de niveaux de 1 dans la description'''
    flatten = []
    for line in description :
        flatten += line
    return flatten


def generateSkipGramdata(descriptions, context_size = 5) :
    '''descriptions = [[str]]'''
    data = []
    for description in descriptions :
        lines = flatten(description)
        line =  flatten(lines)
        line = ['SOS' for i in range(context_size)] + line + ['EOS' for i in range(context_size)] 
        for i in range(context_size, len(line) - context_size):
            context = line[i-context_size : i+context_size+1]
            target = [line[i]]
            data.append((target, context))
    return data

In [137]:
Ngrams = generateSkipGramdata(dialogues_Master, context_size = 5)

**Model**

In [19]:
skipgram = SkipGram(device, 
                    lang = lang_M2DS, 
                    target_size = 11, 
                    embedding_dim = 75, 
                    hidden_dim = 150)
skipgram = skipgram.to(device)

In [91]:
skipgram.load_state_dict(torch.load('C:\\Users\\Jb\\Desktop\\NLP\\saves\\models\\M2DS_2019_10 skipgram_75.pth'))

**Training**

In [154]:
word_embedding_trainer = WordEmbeddingTrainer(device = device)

In [160]:
# 75
for alpha in [0.025, 0.01, 0.005] :
    word_embedding_trainer.train(skipgram, Ngrams, n_iters = 100000, learning_rate=alpha)

0m 0s (- 9m 7s) (100 0%) 7.0563 10.72
0m 1s (- 8m 59s) (200 0%) 6.7815 10.51
0m 1s (- 9m 3s) (300 0%) 6.5769 10.58
0m 2s (- 8m 59s) (400 0%) 6.3487 10.46
0m 2s (- 9m 0s) (500 0%) 6.1837 10.56
0m 3s (- 9m 1s) (600 0%) 6.0625 10.41
0m 3s (- 9m 2s) (700 0%) 5.9279 10.34
0m 4s (- 9m 0s) (800 0%) 5.8341 10.37
0m 4s (- 8m 59s) (900 0%) 5.7745 10.26
0m 5s (- 8m 57s) (1000 1%) 5.7394 10.30
0m 5s (- 8m 56s) (1100 1%) 5.6550 10.44
0m 6s (- 8m 54s) (1200 1%) 5.6022 10.16
0m 7s (- 8m 53s) (1300 1%) 5.6510 10.42
0m 7s (- 8m 58s) (1400 1%) 5.5782 10.05
0m 8s (- 8m 57s) (1500 1%) 5.4799 10.07
0m 8s (- 8m 56s) (1600 1%) 5.5384 10.20
0m 9s (- 8m 54s) (1700 1%) 5.4417 10.14
0m 9s (- 8m 53s) (1800 1%) 5.3675 10.06
0m 10s (- 8m 52s) (1900 1%) 5.4467 10.10
0m 10s (- 8m 52s) (2000 2%) 5.3845 10.02
0m 11s (- 8m 51s) (2100 2%) 5.4491 9.94
0m 11s (- 8m 51s) (2200 2%) 5.3759 9.98
0m 12s (- 8m 50s) (2300 2%) 5.4092 9.96
0m 13s (- 8m 49s) (2400 2%) 5.2512 9.88
0m 13s (- 8m 51s) (2500 2%) 5.3747 9.96
0m 14s (- 8m 

1m 50s (- 7m 13s) (20300 20%) 4.2279 8.55
1m 50s (- 7m 12s) (20400 20%) 4.1410 8.51
1m 51s (- 7m 12s) (20500 20%) 4.3156 8.55
1m 52s (- 7m 11s) (20600 20%) 4.3039 8.43
1m 52s (- 7m 11s) (20700 20%) 4.2531 8.42
1m 53s (- 7m 10s) (20800 20%) 4.2334 8.66
1m 53s (- 7m 10s) (20900 20%) 4.2651 8.45
1m 54s (- 7m 9s) (21000 21%) 4.2137 8.43
1m 54s (- 7m 8s) (21100 21%) 4.3452 8.83
1m 55s (- 7m 8s) (21200 21%) 4.2275 8.50
1m 55s (- 7m 7s) (21300 21%) 4.1339 8.20
1m 56s (- 7m 7s) (21400 21%) 4.0462 8.24
1m 56s (- 7m 6s) (21500 21%) 4.2822 8.51
1m 57s (- 7m 6s) (21600 21%) 4.1819 8.61
1m 57s (- 7m 5s) (21700 21%) 4.4891 8.51
1m 58s (- 7m 5s) (21800 21%) 4.2159 8.11
1m 59s (- 7m 4s) (21900 21%) 4.3009 8.58
1m 59s (- 7m 3s) (22000 22%) 4.2640 8.49
2m 0s (- 7m 3s) (22100 22%) 4.3117 8.41
2m 0s (- 7m 2s) (22200 22%) 4.4866 8.77
2m 1s (- 7m 2s) (22300 22%) 4.1903 8.09
2m 1s (- 7m 1s) (22400 22%) 4.3529 8.66
2m 2s (- 7m 0s) (22500 22%) 3.9297 8.02
2m 2s (- 7m 0s) (22600 22%) 4.2233 8.34
2m 3s (- 6m 59s

3m 42s (- 5m 33s) (40000 40%) 4.0360 8.40
3m 42s (- 5m 32s) (40100 40%) 4.1000 8.45
3m 43s (- 5m 32s) (40200 40%) 3.9376 8.28
3m 43s (- 5m 31s) (40300 40%) 3.9103 8.11
3m 44s (- 5m 30s) (40400 40%) 3.7962 7.99
3m 44s (- 5m 30s) (40500 40%) 4.0100 8.53
3m 45s (- 5m 29s) (40600 40%) 4.0742 8.18
3m 45s (- 5m 29s) (40700 40%) 4.2696 8.68
3m 46s (- 5m 28s) (40800 40%) 3.9990 8.16
3m 47s (- 5m 28s) (40900 40%) 3.9519 8.00
3m 47s (- 5m 27s) (41000 41%) 3.7397 7.85
3m 48s (- 5m 26s) (41100 41%) 3.8401 8.11
3m 48s (- 5m 26s) (41200 41%) 3.8418 7.94
3m 49s (- 5m 25s) (41300 41%) 3.8143 8.01
3m 49s (- 5m 25s) (41400 41%) 3.7504 7.84
3m 50s (- 5m 24s) (41500 41%) 3.7905 8.09
3m 50s (- 5m 24s) (41600 41%) 4.1367 8.51
3m 51s (- 5m 23s) (41700 41%) 3.9638 8.20
3m 52s (- 5m 23s) (41800 41%) 3.7199 7.77
3m 52s (- 5m 22s) (41900 41%) 3.8691 8.10
3m 53s (- 5m 22s) (42000 42%) 4.0301 8.34
3m 54s (- 5m 21s) (42100 42%) 4.0517 8.33
3m 54s (- 5m 21s) (42200 42%) 4.0414 8.40
3m 55s (- 5m 20s) (42300 42%) 3.81

5m 31s (- 3m 43s) (59700 59%) 3.5276 7.58
5m 32s (- 3m 43s) (59800 59%) 3.8216 8.00
5m 32s (- 3m 42s) (59900 59%) 3.8323 8.04
5m 33s (- 3m 42s) (60000 60%) 3.5893 7.86
5m 33s (- 3m 41s) (60100 60%) 3.6462 7.96
5m 34s (- 3m 40s) (60200 60%) 3.6192 7.98
5m 34s (- 3m 40s) (60300 60%) 3.4420 7.41
5m 35s (- 3m 39s) (60400 60%) 3.8181 8.34
5m 35s (- 3m 39s) (60500 60%) 3.7497 8.24
5m 36s (- 3m 38s) (60600 60%) 3.4867 7.55
5m 36s (- 3m 38s) (60700 60%) 3.7837 8.15
5m 37s (- 3m 37s) (60800 60%) 3.7697 7.89
5m 37s (- 3m 36s) (60900 60%) 3.8025 7.94
5m 38s (- 3m 36s) (61000 61%) 3.8558 7.82
5m 38s (- 3m 35s) (61100 61%) 3.9483 8.12
5m 39s (- 3m 35s) (61200 61%) 3.6506 7.78
5m 40s (- 3m 34s) (61300 61%) 3.9396 8.42
5m 40s (- 3m 34s) (61400 61%) 3.7348 7.86
5m 41s (- 3m 33s) (61500 61%) 3.8185 8.38
5m 41s (- 3m 32s) (61600 61%) 3.4829 7.52
5m 42s (- 3m 32s) (61700 61%) 3.5917 7.58
5m 42s (- 3m 31s) (61800 61%) 3.8328 8.26
5m 43s (- 3m 31s) (61900 61%) 3.5787 7.87
5m 43s (- 3m 30s) (62000 62%) 3.74

7m 20s (- 1m 54s) (79400 79%) 3.7231 7.91
7m 21s (- 1m 53s) (79500 79%) 3.3119 7.37
7m 21s (- 1m 53s) (79600 79%) 3.4258 7.81
7m 22s (- 1m 52s) (79700 79%) 3.5314 7.44
7m 22s (- 1m 52s) (79800 79%) 3.5945 7.65
7m 23s (- 1m 51s) (79900 79%) 3.5031 7.56
7m 23s (- 1m 50s) (80000 80%) 3.6539 7.74
7m 24s (- 1m 50s) (80100 80%) 3.4560 7.93
7m 25s (- 1m 49s) (80200 80%) 3.6756 7.95
7m 25s (- 1m 49s) (80300 80%) 3.4048 7.71
7m 26s (- 1m 48s) (80400 80%) 3.2698 7.38
7m 26s (- 1m 48s) (80500 80%) 3.7451 8.01
7m 27s (- 1m 47s) (80600 80%) 3.6537 7.75
7m 27s (- 1m 47s) (80700 80%) 3.5216 7.58
7m 28s (- 1m 46s) (80800 80%) 3.4906 7.29
7m 28s (- 1m 45s) (80900 80%) 3.2502 7.19
7m 29s (- 1m 45s) (81000 81%) 3.5921 7.56
7m 29s (- 1m 44s) (81100 81%) 3.5309 7.79
7m 30s (- 1m 44s) (81200 81%) 3.6498 8.15
7m 30s (- 1m 43s) (81300 81%) 3.9170 8.01
7m 31s (- 1m 43s) (81400 81%) 3.4734 7.20
7m 31s (- 1m 42s) (81500 81%) 3.3219 7.45
7m 32s (- 1m 42s) (81600 81%) 3.3120 7.20
7m 33s (- 1m 41s) (81700 81%) 3.41

...

1m 40s (- 7m 22s) (18500 18%) 3.2546 7.44
1m 40s (- 7m 21s) (18600 18%) 3.4635 8.00
1m 41s (- 7m 20s) (18700 18%) 3.1651 7.36
1m 41s (- 7m 20s) (18800 18%) 3.0546 7.08
1m 42s (- 7m 19s) (18900 18%) 3.0980 7.05
1m 43s (- 7m 19s) (19000 19%) 3.1825 7.52
1m 43s (- 7m 18s) (19100 19%) 2.9433 7.01
1m 44s (- 7m 18s) (19200 19%) 3.1201 7.23
1m 44s (- 7m 17s) (19300 19%) 3.2756 7.62
1m 45s (- 7m 16s) (19400 19%) 3.2026 7.44
1m 45s (- 7m 16s) (19500 19%) 3.2179 7.51
1m 46s (- 7m 15s) (19600 19%) 3.1347 7.48
1m 46s (- 7m 15s) (19700 19%) 3.2842 7.70
1m 47s (- 7m 14s) (19800 19%) 3.2462 7.63
1m 47s (- 7m 13s) (19900 19%) 3.1920 7.24
1m 48s (- 7m 13s) (20000 20%) 3.3274 7.83
1m 48s (- 7m 12s) (20100 20%) 3.1310 7.30
1m 49s (- 7m 12s) (20200 20%) 3.1990 7.69
1m 49s (- 7m 11s) (20300 20%) 3.2095 7.44
1m 50s (- 7m 11s) (20400 20%) 3.1907 7.28
1m 51s (- 7m 10s) (20500 20%) 3.1817 7.35
1m 51s (- 7m 9s) (20600 20%) 3.3005 7.76
1m 52s (- 7m 9s) (20700 20%) 3.1805 7.51
1m 52s (- 7m 8s) (20800 20%) 3.2392 

3m 27s (- 5m 36s) (38200 38%) 3.1288 7.22
3m 28s (- 5m 35s) (38300 38%) 3.0939 7.45
3m 28s (- 5m 35s) (38400 38%) 2.9814 7.15
3m 29s (- 5m 34s) (38500 38%) 3.3555 7.50
3m 30s (- 5m 34s) (38600 38%) 3.1572 7.63
3m 30s (- 5m 33s) (38700 38%) 3.1518 7.49
3m 31s (- 5m 33s) (38800 38%) 3.1606 7.49
3m 31s (- 5m 32s) (38900 38%) 3.2229 7.50
3m 32s (- 5m 32s) (39000 39%) 3.2990 7.34
3m 32s (- 5m 31s) (39100 39%) 3.2124 7.41
3m 33s (- 5m 31s) (39200 39%) 3.2556 7.68
3m 34s (- 5m 30s) (39300 39%) 3.3140 7.77
3m 34s (- 5m 30s) (39400 39%) 3.2213 7.42
3m 35s (- 5m 29s) (39500 39%) 3.2854 7.70
3m 35s (- 5m 29s) (39600 39%) 2.9850 6.77
3m 36s (- 5m 28s) (39700 39%) 2.9343 7.12
3m 36s (- 5m 27s) (39800 39%) 2.9772 7.26
3m 37s (- 5m 27s) (39900 39%) 3.1687 7.44
3m 37s (- 5m 26s) (40000 40%) 3.1090 7.24
3m 38s (- 5m 26s) (40100 40%) 3.2894 7.73
3m 39s (- 5m 25s) (40200 40%) 3.1519 7.43
3m 39s (- 5m 25s) (40300 40%) 3.3599 7.56
3m 40s (- 5m 24s) (40400 40%) 3.2871 7.36
3m 40s (- 5m 24s) (40500 40%) 3.10

5m 16s (- 3m 49s) (57900 57%) 3.3048 7.73
5m 16s (- 3m 49s) (58000 57%) 3.2918 7.58
5m 17s (- 3m 48s) (58100 58%) 3.3134 7.52
5m 17s (- 3m 48s) (58200 58%) 3.1884 7.70
5m 18s (- 3m 47s) (58300 58%) 3.1716 7.15
5m 18s (- 3m 47s) (58400 58%) 3.3361 7.54
5m 19s (- 3m 46s) (58500 58%) 3.1724 7.01
5m 19s (- 3m 45s) (58600 58%) 3.2709 7.65
5m 20s (- 3m 45s) (58700 58%) 3.1561 7.44
5m 20s (- 3m 44s) (58800 58%) 3.1899 7.44
5m 21s (- 3m 44s) (58900 58%) 3.4129 7.68
5m 22s (- 3m 43s) (59000 59%) 3.0973 7.28
5m 22s (- 3m 43s) (59100 59%) 3.2187 7.74
5m 23s (- 3m 42s) (59200 59%) 3.2597 7.49
5m 23s (- 3m 42s) (59300 59%) 3.1312 7.53
5m 24s (- 3m 41s) (59400 59%) 3.1715 7.33
5m 24s (- 3m 41s) (59500 59%) 3.2913 7.76
5m 25s (- 3m 40s) (59600 59%) 3.2755 7.54
5m 25s (- 3m 39s) (59700 59%) 3.4113 7.78
5m 26s (- 3m 39s) (59800 59%) 3.1935 7.59
5m 26s (- 3m 38s) (59900 59%) 3.2012 7.52
5m 27s (- 3m 38s) (60000 60%) 3.1409 7.09
5m 27s (- 3m 37s) (60100 60%) 3.1411 7.40
5m 28s (- 3m 37s) (60200 60%) 3.25

7m 1s (- 2m 1s) (77600 77%) 3.2260 7.61
7m 2s (- 2m 1s) (77700 77%) 3.1893 7.48
7m 2s (- 2m 0s) (77800 77%) 2.9463 7.14
7m 3s (- 2m 0s) (77900 77%) 2.9605 6.94
7m 3s (- 1m 59s) (78000 78%) 3.1281 7.15
7m 4s (- 1m 59s) (78100 78%) 3.0625 6.96
7m 4s (- 1m 58s) (78200 78%) 3.1524 7.38
7m 5s (- 1m 57s) (78300 78%) 3.0879 7.07
7m 5s (- 1m 57s) (78400 78%) 2.9315 7.16
7m 6s (- 1m 56s) (78500 78%) 3.1297 7.47
7m 7s (- 1m 56s) (78600 78%) 3.0447 7.24
7m 7s (- 1m 55s) (78700 78%) 2.9518 7.01
7m 8s (- 1m 55s) (78800 78%) 2.8414 6.67
7m 8s (- 1m 54s) (78900 78%) 3.1411 7.56
7m 9s (- 1m 54s) (79000 79%) 3.3048 7.60
7m 9s (- 1m 53s) (79100 79%) 3.2688 7.30
7m 10s (- 1m 52s) (79200 79%) 3.5408 8.00
7m 10s (- 1m 52s) (79300 79%) 3.0021 7.24
7m 11s (- 1m 51s) (79400 79%) 2.9727 7.23
7m 11s (- 1m 51s) (79500 79%) 3.3451 7.51
7m 12s (- 1m 50s) (79600 79%) 3.5110 8.21
7m 12s (- 1m 50s) (79700 79%) 3.1386 7.35
7m 13s (- 1m 49s) (79800 79%) 3.1186 7.34
7m 14s (- 1m 49s) (79900 79%) 3.2962 7.39
7m 14s (- 1m

8m 48s (- 0m 14s) (97300 97%) 3.1376 7.25
8m 48s (- 0m 14s) (97400 97%) 2.9365 6.91
8m 49s (- 0m 13s) (97500 97%) 3.0492 7.14
8m 49s (- 0m 13s) (97600 97%) 2.9886 7.22
8m 50s (- 0m 12s) (97700 97%) 3.2194 7.36
8m 50s (- 0m 11s) (97800 97%) 3.0622 7.18
8m 51s (- 0m 11s) (97900 97%) 3.0795 7.46
8m 51s (- 0m 10s) (98000 98%) 3.3172 7.60
8m 52s (- 0m 10s) (98100 98%) 3.2770 7.65
8m 52s (- 0m 9s) (98200 98%) 3.1521 7.29
8m 53s (- 0m 9s) (98300 98%) 3.1798 7.45
8m 54s (- 0m 8s) (98400 98%) 3.2155 7.34
8m 54s (- 0m 8s) (98500 98%) 2.9238 6.89
8m 55s (- 0m 7s) (98600 98%) 3.2711 7.65
8m 55s (- 0m 7s) (98700 98%) 3.0958 7.57
8m 56s (- 0m 6s) (98800 98%) 3.2942 7.82
8m 56s (- 0m 5s) (98900 98%) 3.1308 7.49
8m 57s (- 0m 5s) (99000 99%) 3.0169 7.61
8m 57s (- 0m 4s) (99100 99%) 3.0507 7.10
8m 58s (- 0m 4s) (99200 99%) 3.1117 7.15
8m 58s (- 0m 3s) (99300 99%) 3.3111 7.80
8m 59s (- 0m 3s) (99400 99%) 3.0252 6.92
8m 59s (- 0m 2s) (99500 99%) 3.0778 7.34
9m 0s (- 0m 2s) (99600 99%) 2.9662 6.95
9m 0s (-

In [158]:
#torch.save(skipgram.state_dict(), 'C:\\Users\\Jb\\Desktop\\NLP\\saves\\models\\M2DS_2019_10 skipgram_75.pth')

## 2.2 Contextualized Embedding Models

In [22]:
from chatNLP.modules.Encoder_Words_Recurrent import RecurrentWordsEncoder

### 2.2.1 Contextualized Skip-Gram

#### Model

In [86]:
class ModSkipGram(nn.Module):
    def __init__(self, device, 
                 lang, 
                 target_size, 
                 embedding, 
                 hidden_dim):
        
        super(ModSkipGram, self).__init__()
        self.device = device
        self.lang = lang
        self.output_lang = copy.copy(lang)
        self.output_lang.addSentence(['SOS', 'UNK', 'EOS'])
        self.target_size = target_size
        self.hidden_dim = hidden_dim
        
        # embedding is a lookup table
        if type(embedding) == int :
            self.embedding = nn.Embedding(lang.n_words, embedding) 
            self.context_emb = False
            embedding_dim  = embedding
        # embedding is a contextualization module
        else :
            self.embedding = embedding
            for param in self.embedding.parameters()  : param.requires_grad = False
            self.context_emb = True
            embedding_dim  = embedding.output_dim
        self.linear_1 = nn.Linear(embedding_dim, target_size * hidden_dim)
        self.linear_2 = nn.Linear(hidden_dim, self.output_lang.n_words)
        
        
    def variableFromSentence(self, sentence):
        '''Turn a sentence into a torch variable, containing a list of indices according
           to a given language.'''
        indexes = [self.lang.word2index[word] for word in sentence if word in self.lang.word2index]          
        result = Variable(torch.LongTensor(indexes)).to(self.device)
        return result
    

    def forward(self, sentence):
        input = self.variableFromSentence(sentence).view(-1, 1)           # size = (L, 1)
        if self.context_emb : embed = self.embedding(input)[0].squeeze(1) # size = (L, embedding_dim)
        else                : embed = self.embedding(input)               # size = (L, embedding_dim)
        out = self.linear_1(embed).view((input.size(0), self.target_size, self.hidden_dim))   # size = (L, target_size , hidden_dim)
        out = F.relu(out)                                         # size = (L, target_size , hidden_dim)
        out = self.linear_2(out)                                  # size = (L, target_size , lang.n_words)
        log_probs = F.log_softmax(out)                            # size = (L, target_size , lang.n_words)
        return log_probs

In [117]:
class ModWordEmbeddingTrainer(object):
    def __init__(self, 
                 device,
                 criterion = nn.NLLLoss(reduce = False), #nn.BCEWithLogitsLoss(), #nn.BCELoss(), 
                 optimizer = optim.SGD,
                 ignore_index = None,
                 print_every = 100):
        # relevant quantities
        self.device = device
        self.criterion = criterion
        self.optimizer = optimizer
        self.ignore_index = ignore_index
        self.print_every = print_every

        
    def weightsList(self, n):
        half = max(1, n/2)
        weights = [n**2 -(half-i)**2 for i in range(n)]
        weights = [el/sum(weights) for el in weights]
        return weights

        
    def asMinutes(self, s):
        m = math.floor(s / 60)
        s -= m * 60
        return '%dm %ds' % (m, s)
    
    
    def timeSince(self, since, percent):
        now = time.time()
        s = now - since
        es = s / (percent)
        rs = es - s
        return '%s (- %s)' % (self.asMinutes(s), self.asMinutes(rs))
    
    
    def generateOutput(self, agent, line, context_size) :
        output = []
        padded_line = ['UNK' for i in range(context_size)] + line + ['UNK' for i in range(context_size)] 
        for i in range(context_size, len(padded_line) - context_size):
            context = padded_line[i-context_size : i+context_size+1]
            indices = [agent.lang.word2index[word] for word in context if word in agent.lang.word2index]
            output.append(indices)
        return Variable(torch.LongTensor(output)).to(agent.device)
        
        
    def trainLoop(self, agent, sentence, weights, optimizer, learning_rate, compute_diff):
        """Performs a training loop, with forward pass and backward pass for gradient optimisation."""
        optimizer.zero_grad()
        agent.zero_grad()
        
        log_probs = agent(sentence).permute(1, 2, 0)                                       # size = (target_size , lang.n_words, L)
        target_var = self.generateOutput(agent, sentence, context_size = 5).transpose(0,1) # size = (target_size, L)
        #print(log_probs.size(), target_var.size())
        loss = self.criterion(log_probs, target_var)                                        # size = (target_size, L)
        loss = torch.mean(loss, dim = 1)                                                    # size = (target_size)
        #print(loss.size(), weights.size())
        loss = torch.sum(weights * loss)
        loss_diff = 0
        count = 0
        if compute_diff :
            for j in range(len(sentence)) :
                for i in range(agent.target_size) :
                    topv, topi = log_probs[i, :, j].data.topk(1)
                    ni = topi[0][0]
                    tarni = target_var[i, j].data[0]
                    if tarni != self.ignore_index :
                        if tarni != ni : loss_diff += 1
                        count += 1
        loss.backward()
        optimizer.step()                                
        return loss.data[0], loss_diff/max(1, count)
        
        
    def train(self,
              agent, 
              sentences,
              target_size = 11,
              n_iters = 100,
              n_epochs = None,
              learning_rate=0.01,
              random_state = 42
             ):
        """Performs training over a given dataset and along a specified amount of loops."""
        np.random.seed(random_state)
        start = time.time()
        optimizer = self.optimizer([param for param in agent.parameters() if param.requires_grad == True], lr=learning_rate)
        weights = Variable(torch.FloatTensor(self.weightsList(target_size))).to(self.device)
        print_loss_total = 0  
        print_loss_diff_mots_total = 0
        if n_epochs is None :
            for iter in range(1, n_iters + 1):
                sentence = random.choice(sentences)
                loss, loss_diff_mots = self.trainLoop(agent, 
                                                      sentence, 
                                                      weights, 
                                                      optimizer, 
                                                      learning_rate,
                                                      (iter % (self.print_every) == 0))
                # quantité d'erreurs sur la réponse i
                print_loss_total += loss       
                if iter % (self.print_every) == 0 :
                    print_loss_avg = print_loss_total / self.print_every
                    print_loss_total = 0
                    print('%s (%d %d%%) %.4f %.2f %d' % (self.timeSince(start, iter / n_iters),
                                                 iter, iter / n_iters * 100, 
                                                      print_loss_avg, loss_diff_mots, len(sentence)))
        else :
            for epoch in range(n_epochs):
                print('epoch ' + str(epoch))
                np.random.shuffle(ngrams)
                for sentence in sentences :
                    loss, loss_diff_mots = self.trainLoop(agent, 
                                                          sentence, 
                                                          weights, 
                                                          optimizer, 
                                                          learning_rate,
                                                          (iter % (self.print_every) == 0))

                    # quantité d'erreurs sur la réponse i
                    print_loss_total += loss      

                    if iter % (self.print_every) == 0:
                        print_loss_diff_mots_total += loss_diff_mots 
                        print_loss_avg = print_loss_total / self.print_every
                        print_loss_diff_mots_avg = print_loss_diff_mots_total / self.print_every
                        print_loss_total = 0
                        print_loss_diff_mots_total = 0
                        print('%s (%d %d%%) %.4f %.2f' % (self.timeSince(start, iter / n_iters),
                                                     iter, iter / n_iters * 100, 
                                                          print_loss_avg, print_loss_diff_mots_avg))

#### Training

In [118]:
recc_embedding = RecurrentWordsEncoder(device, 
                                       embedding = skipgram.embedding,  
                                       hidden_dim = 50, 
                                       n_layers = 1, 
                                       dropout = 0.25)

mod_skipgram = ModSkipGram(device, 
                          lang = lang_M2DS, 
                          target_size = 11, 
                          embedding = recc_embedding, 
                          hidden_dim = 150)

mod_skipgram = mod_skipgram.to(device)

mod_embedding_trainer = ModWordEmbeddingTrainer(device = device, 
                                                ignore_index = lang_M2DS.word2index['UNK'],
                                                criterion = nn.NLLLoss(reduce = False, ignore_index = lang_M2DS.word2index['UNK']), 
                                                print_every = 1000)

In [27]:
sentences = [sent for dialogue in dialogues_Master for pair in dialogue for sent in pair]

In [119]:
for alpha in [0.05, 0.025, 0.01] :
    mod_embedding_trainer.train(mod_skipgram, sentences, n_iters = 100000, learning_rate = alpha)

0m 8s (- 14m 14s) (1000 1%) 1.6487 0.99 27
0m 17s (- 14m 1s) (2000 2%) 1.6328 0.97 8
0m 25s (- 13m 47s) (3000 3%) 1.5924 0.94 10
0m 34s (- 13m 37s) (4000 4%) 1.5815 0.64 9
0m 42s (- 13m 27s) (5000 5%) 1.5118 1.00 1
0m 51s (- 13m 20s) (6000 6%) 1.5258 0.91 9
0m 59s (- 13m 13s) (7000 7%) 1.4997 0.76 10
1m 8s (- 13m 3s) (8000 8%) 1.4677 0.83 9
1m 16s (- 12m 56s) (9000 9%) 1.4688 0.92 5
1m 25s (- 12m 47s) (10000 10%) 1.4093 0.86 9
1m 33s (- 12m 37s) (11000 11%) 1.3796 0.91 31
1m 42s (- 12m 28s) (12000 12%) 1.3666 0.88 17
1m 50s (- 12m 21s) (13000 13%) 1.3242 0.91 48
1m 59s (- 12m 14s) (14000 14%) 1.3416 0.96 69
2m 7s (- 12m 5s) (15000 15%) 1.2845 0.91 14
2m 16s (- 11m 54s) (16000 16%) 1.2810 0.78 8
2m 24s (- 11m 45s) (17000 17%) 1.2639 0.68 13
2m 32s (- 11m 36s) (18000 18%) 1.2358 0.91 7
2m 41s (- 11m 27s) (19000 19%) 1.2292 0.86 9
2m 49s (- 11m 19s) (20000 20%) 1.2096 0.88 8
2m 58s (- 11m 10s) (21000 21%) 1.2480 0.75 16
3m 6s (- 11m 1s) (22000 22%) 1.2186 0.76 8
3m 15s (- 10m 53s) (23000 

KeyboardInterrupt: 

In [120]:
torch.save(mod_skipgram.state_dict(), 'C:\\Users\\Jb\\Desktop\\NLP\\saves\\models\\M2DS_2019_10 recc_skipgram_75.pth')

***
# 4 + 5 + 6 Chatbot implementation

In [30]:
from chatNLP.models.Chatbot import Chatbot, CreateBot, BotTrainer

## 6.1 Data

In [31]:
dialogues_Master_var = variableFromAllDialogues(lang_M2DS, dialogues_Master, rand = 0)

In [32]:
dialogues_Master_test_var = variableFromAllDialogues(lang_M2DS, dialogues_Master_test, rand = 0)

In [33]:
bound = max([max([len(el[1]) for el in dial]) for dial in dialogues_Master_var])
bound

70

## 6.2 Model

In [34]:
trainer = BotTrainer(device = device,
                     criterion = nn.NLLLoss(), 
                     optimizer = optim.SGD,
                     #autoencoding_ratio = 0.5,
                     clipping = 10,
                     print_every = 100)

In [161]:
chatbot = CreateBot(  lang_M2DS,
                      embedding = skipgram.embedding,
                      hidden_dim = 75,
                      n_layers = 1, # doit être gardé à 1
                    
                      sentence_hidden_dim = 75,
                      hops = 3,
                      share = True, #False
                      transf = True,
                      dropout = 0.10,
                    
                      attn_decoder_n_layers = -1, # -1 : pas d'attention durant le décodage
                      language_model_n_layers = 0,
                      tf_ratio = 1,
                      bound = 75,
                      autoencoding = False,
                    
                      device = device
                     )

chatbot.nbParametres()

649298

In [20]:
chatbot.load_state_dict(torch.load('C:\\Users\\Jb\\Desktop\\NLP\\saves\\models\\M2DS_2019_10 chatbot_100.pth')) # chatbot_original / 26 / 39

In [91]:
# si on veut voir le pipeline du modèle
chatbot

Chatbot(
  (encoder): RecurrentWordsEncoder(
    (embedding): Embedding(1298, 75)
    (dropout): Dropout(p=0.1)
    (bigru): GRU(75, 75, bidirectional=True)
  )
  (attention): RecurrentHierarchicalAttention(
    (dropout): Dropout(p=0.1)
    (attn1): ModuleList(
      (0): AdditiveAttention(
        (attn_layer): Linear(in_features=300, out_features=150, bias=True)
        (attn_v): Linear(in_features=150, out_features=1, bias=False)
      )
      (1): AdditiveAttention(
        (attn_layer): Linear(in_features=300, out_features=150, bias=True)
        (attn_v): Linear(in_features=150, out_features=1, bias=False)
      )
      (2): AdditiveAttention(
        (attn_layer): Linear(in_features=300, out_features=150, bias=True)
        (attn_v): Linear(in_features=150, out_features=1, bias=False)
      )
    )
    (bigru): GRU(150, 75, bidirectional=True)
    (attn2): ModuleList(
      (0): AdditiveAttention(
        (attn_layer): Linear(in_features=300, out_features=150, bias=True)
      

## 6.3 Training

### 6.3.1 Standard training

In [162]:
chatbot.train()
trainer.train(chatbot, dialogues_Master_var, n_iters = 20000, per_dialogue = True, learning_rate=0.05)

0m 26s (- 87m 29s) (100 0%) 5.9005 14.97
0m 51s (- 84m 34s) (200 1%) 5.0519 12.86
1m 12s (- 79m 47s) (300 1%) 4.0959 11.67
1m 35s (- 77m 46s) (400 2%) 3.7418 10.25
1m 56s (- 75m 38s) (500 2%) 3.5851 10.41
2m 18s (- 74m 27s) (600 3%) 3.2492 9.83
2m 41s (- 74m 7s) (700 3%) 2.8933 9.67
2m 58s (- 71m 18s) (800 4%) 2.5445 7.56
3m 24s (- 72m 12s) (900 4%) 2.9038 9.80
3m 44s (- 71m 14s) (1000 5%) 2.8018 9.53
4m 8s (- 71m 13s) (1100 5%) 2.4648 7.12
4m 32s (- 71m 6s) (1200 6%) 2.2838 7.82
4m 57s (- 71m 14s) (1300 6%) 2.3536 8.64
5m 17s (- 70m 21s) (1400 7%) 1.9973 7.40
5m 41s (- 70m 8s) (1500 7%) 1.8539 6.07
6m 3s (- 69m 41s) (1600 8%) 2.4280 8.66
6m 26s (- 69m 22s) (1700 8%) 1.7722 7.29
6m 51s (- 69m 17s) (1800 9%) 1.8064 6.38
7m 10s (- 68m 18s) (1900 9%) 1.7065 6.00
7m 32s (- 67m 56s) (2000 10%) 2.0027 6.61
7m 56s (- 67m 45s) (2100 10%) 2.1251 8.71
8m 15s (- 66m 49s) (2200 11%) 1.5636 4.98
8m 36s (- 66m 16s) (2300 11%) 1.6511 6.81
9m 5s (- 66m 38s) (2400 12%) 1.7121 6.92
9m 25s (- 65m 56s) (2

73m 26s (- 2m 39s) (19300 96%) 0.1212 0.43
73m 52s (- 2m 17s) (19400 97%) 0.1565 1.07
74m 18s (- 1m 54s) (19500 97%) 0.1438 0.50
74m 40s (- 1m 31s) (19600 98%) 0.1130 0.37
75m 5s (- 1m 8s) (19700 98%) 0.1497 0.52
75m 31s (- 0m 45s) (19800 99%) 0.1726 0.68
75m 55s (- 0m 22s) (19900 99%) 0.1927 0.66
76m 21s (- 0m 0s) (20000 100%) 0.1213 0.41


In [163]:
torch.save(chatbot.state_dict(), 'C:\\Users\\Jb\\Desktop\\NLP\\saves\\models\\M2DS_2019_10 chatbot_75_75_75_010_1.pth')

### 6.3.2 Teacher Forcing Ratio = 0.5 + Random masking 25 % of corpus words

In [165]:
dialogues_Master_var_025 = variableFromAllDialogues(lang_M2DS, dialogues_Master, rand = 0.25)

In [164]:
chatbot.decoder.tf_ratio = 0.5

In [ ]:
chatbot.train()
trainer.train(chatbot, dialogues_Master_var_025, n_iters = 20000, per_dialogue = True, learning_rate=0.05)

0m 19s (- 66m 2s) (100 0%) 4.2758 5.20
0m 43s (- 71m 58s) (200 1%) 4.0505 5.40
1m 3s (- 68m 57s) (300 1%) 3.7673 5.72
1m 23s (- 68m 35s) (400 2%) 3.8196 4.51
1m 46s (- 69m 18s) (500 2%) 4.6616 7.74
2m 9s (- 69m 32s) (600 3%) 3.0764 4.45
2m 27s (- 67m 50s) (700 3%) 3.5026 5.00
2m 47s (- 67m 10s) (800 4%) 4.3902 6.61
3m 8s (- 66m 33s) (900 4%) 3.1034 4.78
3m 34s (- 67m 58s) (1000 5%) 3.4157 6.61
3m 58s (- 68m 14s) (1100 5%) 1.9761 2.96
4m 22s (- 68m 24s) (1200 6%) 2.8001 5.20
4m 46s (- 68m 35s) (1300 6%) 2.4998 4.16
5m 9s (- 68m 30s) (1400 7%) 3.7271 4.96
5m 31s (- 68m 14s) (1500 7%) 1.9842 3.35
5m 47s (- 66m 35s) (1600 8%) 3.0697 5.34
6m 8s (- 66m 4s) (1700 8%) 2.6772 3.99
6m 28s (- 65m 27s) (1800 9%) 3.6316 5.09
6m 50s (- 65m 10s) (1900 9%) 3.3142 6.50
7m 11s (- 64m 41s) (2000 10%) 2.9263 4.18
7m 37s (- 65m 3s) (2100 10%) 2.5278 5.35
8m 0s (- 64m 48s) (2200 11%) 3.4682 5.19
8m 22s (- 64m 30s) (2300 11%) 2.9851 5.80
8m 48s (- 64m 34s) (2400 12%) 3.0013 5.35
9m 10s (- 64m 13s) (2500 12%)

In [ ]:
torch.save(chatbot.state_dict(), 'C:\\Users\\Jb\\Desktop\\NLP\\saves\\models\\M2DS_2019_10 chatbot_75_75_75_010_2.pth')

In [ ]:
chatbot.eval()
z = trainer.ErrorCount(chatbot, dialogues_Master_test_var) # erreurs par réponse

### 6.3.3 Learning rate harved

In [ ]:
chatbot.train()
trainer.train(chatbot, dialogues_Master_var_025, n_iters = 20000, per_dialogue = True, learning_rate=0.025)

In [ ]:
# 12241 / 12890
torch.save(chatbot.state_dict(), 'C:\\Users\\Jb\\Desktop\\NLP\\saves\\models\\M2DS_2019_10 chatbot_75_75_75_010_3.pth')

In [ ]:
chatbot.eval()
z = trainer.ErrorCount(chatbot, dialogues_Master_test_var) # erreurs par réponse

### 6.3.4 Teacher Forcing Ratio = 0.25 + Learning Rate harved + Random masking 35 % of corpus words


In [ ]:
dialogues_Master_var_035 = variableFromAllDialogues(lang_M2DS, dialogues_Master, rand = 0.35)

In [ ]:
chatbot.decoder.tf_ratio = 0.25

In [ ]:
chatbot.train()
trainer.train(chatbot, dialogues_Master_var_035, n_iters = 30000, per_dialogue = True, learning_rate=0.01)

In [ ]:
torch.save(chatbot.state_dict(), 'C:\\Users\\Jb\\Desktop\\NLP\\saves\\models\\M2DS_2019_10 chatbot_75_75_75_010_4.pth')

In [ ]:
# 12749 / 13547 / 13202
chatbot.eval()
z = trainer.ErrorCount(chatbot, dialogues_Master_test_var) # erreurs par réponse (19854 au total)

In [123]:
chatbot.train()
trainer.train(chatbot, dialogues_Master_var_035, n_iters = 30000, per_dialogue = True, learning_rate=0.005)

0m 18s (- 92m 4s) (100 0%) 1.6204 2.70
0m 35s (- 88m 35s) (200 0%) 1.5671 2.78
0m 55s (- 91m 35s) (300 1%) 2.4450 4.61
1m 11s (- 88m 45s) (400 1%) 2.0794 4.08
1m 32s (- 90m 39s) (500 1%) 1.0823 2.56
1m 56s (- 94m 48s) (600 2%) 1.7909 3.57
2m 22s (- 99m 21s) (700 2%) 1.4156 2.68
2m 47s (- 102m 5s) (800 2%) 1.4657 2.78
3m 11s (- 103m 19s) (900 3%) 2.0870 3.67
3m 32s (- 102m 55s) (1000 3%) 1.9086 3.03
3m 56s (- 103m 37s) (1100 3%) 1.8028 3.96
4m 18s (- 103m 35s) (1200 4%) 2.0108 4.48
4m 43s (- 104m 9s) (1300 4%) 1.0788 3.29
5m 6s (- 104m 27s) (1400 4%) 1.3140 2.61
5m 27s (- 103m 49s) (1500 5%) 1.4665 3.00
5m 52s (- 104m 8s) (1600 5%) 2.4762 3.88
6m 15s (- 104m 7s) (1700 5%) 1.4312 2.95
6m 37s (- 103m 44s) (1800 6%) 1.9163 4.44
6m 57s (- 102m 58s) (1900 6%) 1.4971 3.62
7m 12s (- 100m 48s) (2000 6%) 1.4926 2.65
7m 33s (- 100m 24s) (2100 7%) 0.9639 2.03
7m 53s (- 99m 49s) (2200 7%) 1.2820 2.85
8m 18s (- 99m 58s) (2300 7%) 0.9479 1.70
8m 36s (- 99m 4s) (2400 8%) 2.1251 4.90
8m 59s (- 98m 54s)

70m 43s (- 39m 12s) (19300 64%) 1.4100 3.52
71m 11s (- 38m 53s) (19400 64%) 1.8220 4.68
71m 35s (- 38m 33s) (19500 65%) 1.8254 3.35
71m 55s (- 38m 10s) (19600 65%) 2.0955 3.88
72m 13s (- 37m 45s) (19700 65%) 2.1306 4.08
72m 34s (- 37m 23s) (19800 66%) 1.4087 2.75
72m 57s (- 37m 1s) (19900 66%) 1.8329 4.25
73m 19s (- 36m 39s) (20000 66%) 1.9546 3.55
73m 39s (- 36m 16s) (20100 67%) 1.5573 3.56
73m 59s (- 35m 53s) (20200 67%) 1.4064 3.21
74m 19s (- 35m 31s) (20300 67%) 1.7159 3.28
74m 38s (- 35m 7s) (20400 68%) 1.2947 2.97
74m 57s (- 34m 44s) (20500 68%) 1.5853 4.04
75m 17s (- 34m 21s) (20600 68%) 1.3911 3.00
75m 39s (- 33m 59s) (20700 69%) 1.5303 2.94
75m 58s (- 33m 36s) (20800 69%) 1.3243 3.26
76m 17s (- 33m 12s) (20900 69%) 1.3355 2.38
76m 37s (- 32m 50s) (21000 70%) 2.6029 4.91
76m 59s (- 32m 28s) (21100 70%) 2.2019 4.41
77m 23s (- 32m 7s) (21200 70%) 1.2531 2.60
77m 46s (- 31m 46s) (21300 71%) 1.7031 4.51
78m 10s (- 31m 25s) (21400 71%) 1.0257 2.75
78m 29s (- 31m 1s) (21500 71%) 1.32

In [124]:
torch.save(chatbot.state_dict(), 'C:\\Users\\Jb\\Desktop\\NLP\\saves\\models\\M2DS_2019_10 chatbot_75_75_75_010_6.pth')

In [125]:
# 12759 / 13849 / 13000
chatbot.eval()
z = trainer.ErrorCount(chatbot, dialogues_Master_test_var) # erreurs par réponse

The repartition of errors : [13000, 9, 57, 88, 4, 63, 8, 50, 150, 163, 440, 442, 707, 1032, 756, 649, 715, 547, 441, 200, 106, 147, 0, 22, 279, 24, 164, 52, 4, 49, 1, 3, 1, 10, 0, 22, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 28, 0, 0, 0, 0, 0, 0, 0, 0, 5, 15, 0, 0, 0, 0, 0, 0, 0, 0, 105]


## 6.4 test

### 6.4.1 Error counts



In [38]:
chatbot.eval()
z = trainer.ErrorCount(chatbot, dialogues_Master_test_var) # erreurs par réponse

The repartition of errors : [12759, 0, 68, 0, 0, 29, 0, 96, 54, 113, 645, 395, 284, 591, 971, 902, 577, 1111, 136, 330, 143, 281, 0, 0, 67, 108, 47, 43, 1, 35, 0, 0, 0, 0, 0, 16, 0, 5, 0, 5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 7, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 35]


### 6.4.2 Visualization of attention

In [52]:
def showAnswer(model, dialogue):
    answer, log_probas, attn1_w, attn2_w, _ = model.answerTrain(dialogue)
    dialogue = model.flattenDialogue(dialogue)
    print('User : ', ' '.join([model.lang.index2word[int(word)] for word in dialogue[-2][:-1]]))
    print('target : ', ' '.join([model.lang.index2word[int(word)] for word in dialogue[-1][:-1]]))
    print('predic : ', ' '.join([model.lang.index2word[int(word)] for word in answer]))
    return

In [53]:
chatbot.eval()
n_errors = 14
i = 724
dialogue = dialogues_Master_test_var[z[n_errors][i][0]][:z[n_errors][i][1]+1]
#chatbot.showAttention(dialogue)
showAnswer(chatbot, dialogue)

User :  oui , j ' ai etudie a unk_token
target :  etes - vous un etudiant inscrit a lyon 1 ?
predic :  il faut que vous deposiez votre candidature directement aupres de campus france .


In [37]:
chatbot.eval()
i = 10
dialogue = dialogues_Master_test_var[i][:-3]
chatbot.showAttention(dialogue)

ValueError: too many values to unpack (expected 4)

### 6.4.3 Interactive mode

write **quit** or **q** or **eoc** (end of conversation) to close conversation interface :

In [21]:
def ReplaceMotVar(motsVar, raw_sentence):
    sentence = []
    word_list = raw_sentence.split(' ')
    for word in word_list :
        if word in motsVar.keys() :
            sentence.append(motsVar[word])
        else :
            sentence.append(word)
    return ' '.join(sentence)


def repair(sentence) :
    s = re.sub(" ' ", "'", sentence)
    s = re.sub(" - ", "-", s)
    s = re.sub(" ,", ",", s)
    s = re.sub(r'(?<=\d) \. (?=\d)', '.', s)
    s = re.sub(" \.", ".", s)
    s = re.sub("\( ", "(", s)
    s = re.sub(" \)", ")", s)
    s = s[0].upper() + s[1:]
    return s


def InteractiveEvaluation(agent) :
    agent.initMemory()
    while True :
        text = input()
        #print('User : {}'.format(text))
        if text == 'eoc' or text == 'q' or text == 'quit' : 
            break
        reponse, attn1_weights, attn2_weights = agent(text)
        reponse = ReplaceMotVar(motsVar, reponse)
        reponse = repair(reponse)
        print('Bot  : {}'.format(reponse))

In [128]:
chatbot.eval()
InteractiveEvaluation(chatbot)

comment on peut payer ?
Bot  : Vous pouvez payer par carte, par cheque bancaire, par virement ou en espece.
quels sont les modes de payement
Bot  : Le stage doit avoir une duree de 4 a 6 mois.
combien y a - t - il d ' ects par cours ?
Bot  : Il y a en tout 400h de cours.
combien d'etcs
Bot  : Il y a en tout 400h de cours.
le nombre d'etcs
Bot  : D'accord, comment puis-je vous aider ?
le nombre d etcs est de combien ?
Bot  : Je suis la pour vous renseigner sur le master 2 data science.
etcs
Bot  : Avez-vous d'autres questions ?
oui, est ce que c'est difficile ?
Bot  : Non, mais il faut travailler assidument.
ca veur dire qu'il faut travailler jours et nuits ?
Bot  : Il faut que vous deposiez votre candidature directement aupres de campus france.
quit


In [22]:
chatbot.eval()
InteractiveEvaluation(chatbot)

salut toi
Bot  : Bonjour, comment puis-je vous aider ?
comment tu te nomme ?
Bot  : Je suis la pour vous renseigner sur le master 2 data science.
quel est ton nom ?
Bot  : Je m'appelle Horace.
cool enchanté horace
Bot  : Bonjour, comment puis-je vous aider ?
what yu thking know
Bot  : Je vous en prie
ta gueule
Bot  : D'accord, comment puis-je vous aider ?
quit


# 7 Noise filtering

## 7.1 Noisy data generation

In [23]:
random.seed(42)

In [24]:
def randomMask(clean_sentences, rand = 0.2) :
    def choseWord(word) :
        return word if random.random() >= rand else 'UNK'
    masked_sentences = []
    for sentence in clean_sentences :
        words = [choseWord(word) for word in sentence.split()]
        masked_sentences.append(' '.join(words))
    return masked_sentences



def randomNoise(clean_sentences, lang, prop = 1.0):
    noisy_sentences = []
    for sentence in clean_sentences :
        if len(sentence.split()) >= 5 :
            words = [lang.index2word[random.randint(3, lang.n_words-1)] for _ in sentence.split()]
            words = [word for i, word in enumerate(words) if i < 5 or random.random() < prop]
            rand_unk = random.randint(1, len(words)-1)
            words.insert(rand_unk, 'UNK')
            if rand_unk > len(words)/ 2 : words += random.choice(['?', '.'])
            noisy_sentences.append(' '.join(words))
    return noisy_sentences

In [25]:
clean_sentences = [sentence for sentence in list(set([' '.join(pair[0]) for diag in dialogues_Master for pair in diag]))]
print(len(clean_sentences))

masked_sentences_000 = []
masked_sentences_015 = []
masked_sentences_025 = []
noisy_sentences_100 = []
noisy_sentences_050 = []

for _ in range(10) :
    masked_sentences_000 += clean_sentences
    masked_sentences_015 += randomMask(clean_sentences, rand = 0.15)
    masked_sentences_025 += randomMask(clean_sentences, rand = 0.25)

    noisy_sentences_100 += randomNoise(clean_sentences, lang_M2DS, prop = 1.0)
    noisy_sentences_050 += randomNoise(clean_sentences, lang_M2DS, prop = 0.5)

1364


In [26]:
discriminator_dataset = [[sentence, 1] for sentence in masked_sentences_000 + masked_sentences_015 + masked_sentences_025] + \
                        [[sentence, 0] for sentence in noisy_sentences_100 + noisy_sentences_050]
random.shuffle(discriminator_dataset)
len(discriminator_dataset)

66900

In [27]:
discriminator_dataset[7][0]

'les cours sont UNK memes entre UNK informatique UNK UNK ?'

## 7.2 Noise filtering model

In [28]:
from chatNLP.utils.Noise_Filter import NoiseFilter, NoiseFilterTrainer, NoiseFilterWrapper

In [29]:
noise_filter = NoiseFilter(chatbot, pretrained = True, layers = [50, 50]).to(device)

In [30]:
noise_filter.nbParametres()

10202

In [ ]:
#noise_filter.load_state_dict(torch.load('C:\\Users\\Jb\\Desktop\\NLP\\saves\\models\\M2DS_2019_final_noise_filter.pth'))

In [34]:
noise_filter_trainer = NoiseFilterTrainer(device,
                                          criterion = nn.NLLLoss(), 
                                          optimizer = optim.SGD)

In [35]:
noise_filter.train()
noise_filter_trainer.train(noise_filter, sentences = discriminator_dataset, n_iters = 20000, learning_rate=0.01)

0m 1s (- 4m 30s) (100 0%) 0.7033 0.58
0m 2s (- 3m 36s) (200 1%) 0.6104 0.28
0m 3s (- 3m 17s) (300 1%) 0.6632 0.47
0m 3s (- 3m 7s) (400 2%) 0.5668 0.39
0m 4s (- 3m 1s) (500 2%) 0.6100 0.44
0m 5s (- 2m 56s) (600 3%) 0.6134 0.43
0m 6s (- 2m 52s) (700 3%) 0.5791 0.38
0m 7s (- 2m 49s) (800 4%) 0.5662 0.31
0m 7s (- 2m 47s) (900 4%) 0.5960 0.40
0m 8s (- 2m 45s) (1000 5%) 0.5095 0.29
0m 9s (- 2m 43s) (1100 5%) 0.4927 0.26
0m 10s (- 2m 41s) (1200 6%) 0.5237 0.32
0m 11s (- 2m 39s) (1300 6%) 0.4389 0.21
0m 11s (- 2m 38s) (1400 7%) 0.4833 0.31
0m 12s (- 2m 38s) (1500 7%) 0.3968 0.23
0m 13s (- 2m 36s) (1600 8%) 0.4835 0.28
0m 14s (- 2m 35s) (1700 8%) 0.4663 0.27
0m 15s (- 2m 34s) (1800 9%) 0.4843 0.26
0m 16s (- 2m 33s) (1900 9%) 0.4760 0.29
0m 16s (- 2m 31s) (2000 10%) 0.5011 0.31
0m 17s (- 2m 31s) (2100 10%) 0.4996 0.33
0m 18s (- 2m 32s) (2200 11%) 0.5167 0.30
0m 19s (- 2m 32s) (2300 11%) 0.5095 0.27
0m 20s (- 2m 31s) (2400 12%) 0.5487 0.33
0m 21s (- 2m 30s) (2500 12%) 0.4961 0.26
0m 22s (- 2m 28s

In [37]:
noise_filter.train()
noise_filter_trainer.train(noise_filter, sentences = discriminator_dataset, n_iters = 10000, learning_rate=0.005)
noise_filter_trainer.train(noise_filter, sentences = discriminator_dataset, n_iters = 30000, learning_rate=0.001)

0m 1s (- 1m 55s) (100 1%) 0.3162 0.15
0m 2s (- 1m 42s) (200 2%) 0.2730 0.18
0m 2s (- 1m 33s) (300 3%) 0.2949 0.19
0m 3s (- 1m 28s) (400 4%) 0.2548 0.16
0m 4s (- 1m 28s) (500 5%) 0.2382 0.16
0m 5s (- 1m 25s) (600 6%) 0.2973 0.14
0m 6s (- 1m 23s) (700 7%) 0.2080 0.12
0m 7s (- 1m 21s) (800 8%) 0.3292 0.17
0m 7s (- 1m 19s) (900 9%) 0.2538 0.14
0m 8s (- 1m 18s) (1000 10%) 0.4207 0.17
0m 9s (- 1m 16s) (1100 11%) 0.2301 0.10
0m 10s (- 1m 15s) (1200 12%) 0.3076 0.19
0m 11s (- 1m 14s) (1300 13%) 0.2710 0.17
0m 11s (- 1m 13s) (1400 14%) 0.3855 0.19
0m 12s (- 1m 11s) (1500 15%) 0.1809 0.08
0m 13s (- 1m 10s) (1600 16%) 0.1815 0.11
0m 14s (- 1m 9s) (1700 17%) 0.2196 0.14
0m 15s (- 1m 9s) (1800 18%) 0.3264 0.17
0m 16s (- 1m 8s) (1900 19%) 0.2728 0.17
0m 16s (- 1m 7s) (2000 20%) 0.2735 0.20
0m 17s (- 1m 6s) (2100 21%) 0.3328 0.16
0m 18s (- 1m 5s) (2200 22%) 0.2397 0.16
0m 19s (- 1m 4s) (2300 23%) 0.2542 0.16
0m 20s (- 1m 3s) (2400 24%) 0.2971 0.16
0m 20s (- 1m 2s) (2500 25%) 0.1961 0.14
0m 21s (- 1m 

1m 29s (- 2m 45s) (10500 35%) 0.2814 0.22
1m 30s (- 2m 44s) (10600 35%) 0.1586 0.10
1m 30s (- 2m 44s) (10700 35%) 0.2575 0.14
1m 31s (- 2m 43s) (10800 36%) 0.2443 0.15
1m 32s (- 2m 42s) (10900 36%) 0.2133 0.12
1m 33s (- 2m 41s) (11000 36%) 0.2571 0.13
1m 34s (- 2m 40s) (11100 37%) 0.3312 0.21
1m 34s (- 2m 39s) (11200 37%) 0.3057 0.20
1m 35s (- 2m 38s) (11300 37%) 0.1807 0.11
1m 36s (- 2m 37s) (11400 38%) 0.2187 0.15
1m 37s (- 2m 36s) (11500 38%) 0.2628 0.14
1m 38s (- 2m 35s) (11600 38%) 0.2337 0.12
1m 39s (- 2m 34s) (11700 39%) 0.2829 0.20
1m 39s (- 2m 34s) (11800 39%) 0.2000 0.15
1m 40s (- 2m 33s) (11900 39%) 0.2578 0.15
1m 41s (- 2m 32s) (12000 40%) 0.2358 0.14
1m 42s (- 2m 31s) (12100 40%) 0.2393 0.13
1m 43s (- 2m 30s) (12200 40%) 0.2687 0.17
1m 43s (- 2m 29s) (12300 41%) 0.2644 0.12
1m 44s (- 2m 28s) (12400 41%) 0.2023 0.13
1m 45s (- 2m 27s) (12500 41%) 0.2699 0.23
1m 46s (- 2m 26s) (12600 42%) 0.3215 0.17
1m 47s (- 2m 26s) (12700 42%) 0.3438 0.21
1m 48s (- 2m 25s) (12800 42%) 0.27

In [39]:
torch.save(noise_filter.state_dict(), 'C:\\Users\\Jb\\Desktop\\NLP\\saves\\models\\M2DS_2019_final noise_filter.pth')

## 7.3 Noise filtering integration

model built on top of a pretrained chatbot model

In [ ]:
def SmartInteractiveEvaluation(agent) :
    agent.chatbot.initMemory()
    while True :
        text = input()
        #print('User : {}'.format(text))
        if text == 'eoc' or text == 'q' or text == 'quit' : 
            break
        reponse, attn1_weights, attn2_weights = agent(text)
        reponse = ReplaceMotVar(motsVar, reponse)
        reponse = repair(reponse)
        print('Bot  : {}'.format(reponse))

In [ ]:
noise_filter.eval()
smart_chatbot = NoiseFilterWrapper(noise_filter, chatbot)
smart_chatbot.eval()
SmartInteractiveEvaluation(smart_chatbot)